# From Sparse to Dense representations of text

## Sparse representation of text: One-hot vectors

Traditionally we use a one-hot vectors to represent text as vectors. For example, for a sentence "a quick brown fox", we could end up with following one hot vectors.

    [1, 0, 0, 0] - a
    [0, 1, 0, 0] - quick
    [0, 0, 1, 1] - brown
    [0, 0, 0, 1] - fox
    
It's easy to see that for larger vocabularies such as web pages, faq articles, etc. i.e. the kind we typically see in IR settings, these vectors will be very long and very sparse with the dimensionality equal to the vocab size.

These vectors also have no information about the words they represent. For example, here "quick" is as similar to "fox" as it is to the word "brown".

** Vocab size refers to the number of unique tokens/terms used in a collection of documents.

## Distributional Semantics

Formally, dense representation of text is easy to define

$$ f(text) => \!R^n $$

In the above example, if we take n = 5, our dense vectors might look like

    [0.98, -1.12, 0.21, 0.01] - a
    [1.22, 2.12, -3.12, 3.11] - quick
    [-1.23, 4.30, 1.16, 0.90] - brown
    [0.11, 0.12, -1.23, 4.65] - fox

To capture the meaning of words in such dense vectors, we need to consider the notion of context for words in a piece of text. For example, consider these texts:

    1. A bottle of ________ is on the table.
    2. Everyone likes ________.
    3. ________ makes you drunk.
    4. We make ________ out of corn.

Now consider the word that could appear in the blank position. Given the context, we can guess it's an alcoholic beverage (the correct answer is `tezguino` btw).
Other alcoholic drinks made of corn also qualify here. The idea is that we can guess the word based on the it's surrounding words or the context in which it appears.

Techniques like word2vec, LSTMs, transformers build on top of this idea.

## Before Transformers

### Count based methods

The general process of creating dense represenations using count based methods is to:
 1. Create a word-context matrix.
 2. Reduce it's dimensionality.

We reduce dimensionality because of the large vocab size as well as the sparsity of such matrices(most of the terms are 0 in the matrix).
One such technique is LSA(Latent Semantic Analysis) which does SVD on the term document matrix(could be TF_IDF).

![](assets/lsa.jpg)
source: https://www.frontiersin.org/articles/10.3389/fphys.2013.00008/full

### Word2Vec

Word2Vec works on the similar premise of encoding contextual information into the dense representation for a word.
Word2Vec wors in an iterative manner:
 - Start with a huge text corpus.
 - Go over the text using a sliding window. At each step, take a center word i and the context words(within the window).
 - For the center word, compute probabilites for all the words in the vocab using softmax.
 - Computer cross entropy loss for the context words and backprop over the embeddings to refine them.

![](assets/w2v_window.png)

There are two methods used to create the word embeddings:
 - CBOW(Continuous bag of words): Uses the context words to predict the center word.
 - Skip gram: Uses the center word to predict the context words.

![](assets/w2v.png)
source: https://www.researchgate.net/figure/CBOW-and-Skip-Gram-neural-architectures_fig14_325651523

The ojective function is formulated as

$$ J(\theta) = - \frac{1}{T} \sum_{t=1}^T \sum_{-m<=j<=m, j\neq0} P(w_{t+j}|w_t,\theta) $$

𝑃(𝑤𝑡+𝑗|𝑤𝑡,𝜃) - Probability of the context word given center word(skip gram)

Essentially we are going over the whole text using a sliding window and computing the loss using the probability from softmax.

Some of the drawbacks of Wor2Vec:
 - Essentially a bag of words technique as we don't consider the order in which the words appear in a window.
 - Limited window size makes the learned representaions very limited in the sense that they only capture very local contexts.
 - We can't capture multiple embeddings for the same word in different context, for example, "a river bank", "bank deposit". The word "bank" ends with a single embedding even though it means two different things here.
 - Out of vocab words are not handeled.

There are methods to deal with some of these issues, like subword embeddings using fastText can handle out of vocab words. In general these bag of wordd techniques have fallen out of favor for methods we are going to discuss next.

### Recurrent Neural Nets

Before Transformers, RNNs especially LSTMs were all the rage but they have also fallen out of favour because of the following drawbacks:
 - The temporal dependence in updating the hidden state(over the tokens) make then slow to train and not let us effeciently use the modern hardware for training(gpus).
 - Even though theoratically LSTMs can handle arbitrarily long sequences, in practice they suffer from vanishing gradient.
 - The biggest problem is the bottleneck at the end of the sequence where we are trying to store all the historical information from the sequence into one final hidden vector.

The attention mechanism on top of the LSTMs help us solve the final bottleneck issue but we still struggle with slow training times as discussed in the first point.

As Transformers have proven, we don't really need this recurrent mechanism, by adding positional information into each token embedding and doing self attention on top of it, we can learn powerful representations of text and use them for all the downstream NLP tasks.

## Transformers for IR